In [ ]:
!unzip /content/RaFD.zip

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=723970f42c4020862bfc1b1fce3fe74f5dd281532e99abf0b46a144071c6bfbd
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import PIL.Image
import numpy as np
import face_recognition

def crop_to_square_with_face_center(image_path):
    img = PIL.Image.open(image_path)
    img_array = np.array(img)

    face_locations = face_recognition.face_locations(img_array)

    if len(face_locations) > 0:
        # Assuming the first face is the main one
        top, right, bottom, left = face_locations[0]

        face_width = right - left
        face_height = bottom - top
        square_size = max(face_width, face_height)

        left_crop = max(0, left + (face_width - square_size) // 2)
        top_crop = max(0, top + (face_height - square_size) // 2)
        right_crop = min(img.width, left_crop + square_size)
        bottom_crop = min(img.height, top_crop + square_size)

        cropped_img = img.crop((left_crop, top_crop, right_crop, bottom_crop))
        cropped_img = cropped_img.resize((128, 128))
        cropped_img.save(image_path)
        return cropped_img
    else:
        return img



In [ ]:
# YALE dataset
import os
import shutil
import PIL.Image

def reorganize_yale_database(source_folder, destination_folder, subjects, conditions):
    for condition in conditions:
        os.makedirs(os.path.join(destination_folder, 'train', condition), exist_ok=True)
        os.makedirs(os.path.join(destination_folder, 'test', condition), exist_ok=True)

    for subject in subjects:
        for idx, condition in enumerate(conditions):
            source_path = os.path.join(source_folder, f"{subject}.{condition}")
            dest_folder = 'test' if subject.endswith('13') or subject.endswith('15') else 'train'
            dest_path = os.path.join(destination_folder, dest_folder, condition, f"{subject}_{condition}.jpg")
            shutil.copy(source_path, dest_path)
            cropped_image = crop_to_square_with_face_center(dest_path)

# data folder contains 165 images
source_folder = "/content/data"
# RaFD folder contains labeled folders with their specific items
destination_folder = "/content/RaFD/"
subjects = ['subject0' + str(nr) if nr < 10 else 'subject' + str(nr) for nr in range(1, 16)]
conditions = ['centerlight', 'happy', 'glasses', 'leftlight', 'noglasses', 'normal',
              'rightlight', 'sad', 'sleepy', 'surprised', 'wink']

reorganize_yale_database(source_folder, destination_folder, subjects, conditions)


In [ ]:
from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

source_folder = "/content/RaFD/"
destination_folder = "/content/drive/MyDrive/unu/"
shutil.copytree(source_folder, destination_folder)

print("Folder copied to Google Drive successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder copied to Google Drive successfully.
